For the given data in Excel file,
### a. Use matrix operation for parameter identification and BIC
to find the best-fit model for the linear regression model
where are features
### b. Calculate the SSE of the best-fit model

In [1]:
import pandas as pd
import numpy as np
import os

os.chdir("/content/drive/MyDrive/生醫資訊基礎")

# 定義線性迴歸類
class LinearRegression:
    def __init__(self):
        self.coef = None
        self.intercept = None

    def fit(self, x, y):
        # 在 x 前加一columns 1，表示截距
        x = np.hstack([np.ones((x.shape[0], 1)), x])
        beta = (np.linalg.inv(x.T @ x) @ x.T) @ y
        self.intercept = beta[0]
        self.coef = beta[1:]

    def predict(self, x):
        y_pred = self.intercept + x @ self.coef
        return y_pred

    def SSE(self, y_real, y_pred):
        e = y_real - y_pred
        return np.sum(e**2)#(e.T @ e)[0][0]

    def BIC(self, y_real, y_pred, n, k):
        sse = self.SSE(y_real, y_pred)
        return k * np.log(sse/k) + n * np.log(k)

In [2]:
# 讀入檔案
file_path = "20231127 HwData.xlsx"
x = pd.read_excel(file_path, sheet_name="Features", index_col=0).T
y = pd.read_excel(file_path, sheet_name="RiskScore", index_col=0).T

In [3]:
# 對每個模型組合計算BIC和SSE並記錄
n_features = len(x.columns)
result_df = pd.DataFrame(columns=["n", "SSE", "BIC", "mask", "coef", "intercept"], index=range(1, 2**n_features))

for i in range(1, 2**n_features):
    mask_str = bin(i)[2:].zfill(n_features)[::-1]
    mask = np.where(np.array(list(mask_str))=="1", True, False)
    x_combination = x.loc[:, mask].values
    y_real = y.values
    n = x_combination.shape[1]
    k = x_combination.shape[0]
    model = LinearRegression()
    model.fit(x_combination, y_real)
    y_pred = model.predict(x_combination)
    sse = model.SSE(y_real, y_pred)
    bic = model.BIC(y_real, y_pred, n, k)
    result_df.loc[i, :] = n, sse, bic, mask, model.coef, model.intercept

In [4]:
# 印出結果
min_BIC = result_df[result_df.BIC == result_df.BIC.min()]
mask = min_BIC.loc[:, "mask"].values[0]
print(f"Best Fit Model\n{'-'*70}")
print(f"BIC: {min_BIC.BIC.values[0]:.6f}")
print(f"SSE: {min_BIC.SSE.values[0]:.6f}")
print(f"Intercept: {min_BIC.intercept.values[0][0]:.6f}")
print(f"features: {x.columns[mask].values}")
print(f"Coef: {[f'{coef:.6f}' for coef in min_BIC.coef.values[0].flatten()]}")

Best Fit Model
----------------------------------------------------------------------
BIC: -5.960679
SSE: 18.545618
Intercept: -0.601784
features: ['Feature A' 'Feature C' 'Feature E' 'Feature G' 'Feature H']
Coef: ['-0.419937', '1.023581', '1.325989', '-0.298860', '-1.110467']
